In [1]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize
import emoji
import numpy as np

from Lab_support.utils2 import get_dict

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/akshatgupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'
data = re.sub(r'[,!?;-]+', '.', corpus)
data = nltk.word_tokenize(data)
data = [ch.lower() for ch in data
        if ch.isalpha()
        or ch == '.'
        or emoji.get_emoji_regexp().search(ch)]

In [4]:
data

['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.']

In [5]:
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)
    data = [ch.lower() for ch in data
            if ch.isalpha()
            or ch == '.'
            or emoji.get_emoji_regexp().search(ch)]
    return data

In [6]:
corpus = 'I am happy because I am learning'
words = tokenize(corpus)
words

['i', 'am', 'happy', 'because', 'i', 'am', 'learning']

In [7]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [8]:
for x, y in get_windows(words,2):
    print(f'{y}\t{x}')

happy	['i', 'am', 'because', 'i']
because	['am', 'happy', 'i', 'am']
i	['happy', 'because', 'am', 'learning']


In [9]:
word2Ind, Ind2word = get_dict(words)
V = len(word2Ind)

In [10]:
def word_to_one_hot_vector(word, word2Ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1
    
    return one_hot_vector

In [11]:
n = word2Ind['happy']
center_word_vector = word_to_one_hot_vector('happy',word2Ind,V)

In [12]:
center_word_vector

array([0., 0., 1., 0., 0.])

In [13]:
def context_words_to_vector(context_words, word2Ind, V):
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0)
    
    return context_words_vectors

In [14]:
context_words = ['i','am','i','because']
context_words_vector = context_words_to_vector(context_words,word2Ind,V)

In [15]:
context_words_vector

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [16]:
def get_training_example(words, C, word2Ind, V):
    for context_words, center_word in get_windows(words, C):
        yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

In [17]:
def relu(z):
    result = z.copy()
    result[result < 0] = 0
    
    return result

In [18]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z)
    
    return e_z / sum_e_z

In [19]:
N = 3

In [20]:
W1 = np.array([[ 0.41687358,  0.08854191, -0.23495225,  0.28320538,  0.41800106],
               [ 0.32735501,  0.22795148, -0.23951958,  0.4117634 , -0.23924344],
               [ 0.26637602, -0.23846886, -0.37770863, -0.11399446,  0.34008124]])

W2 = np.array([[-0.22182064, -0.43008631,  0.13310965],
               [ 0.08476603,  0.08123194,  0.1772054 ],
               [ 0.1871551 , -0.06107263, -0.1790735 ],
               [ 0.07055222, -0.02015138,  0.36107434],
               [ 0.33480474, -0.39423389, -0.43959196]])

b1 = np.array([[ 0.09688219],
               [ 0.29239497],
               [-0.27364426]])

b2 = np.array([[ 0.0352008 ],
               [-0.36393384],
               [-0.12775555],
               [-0.34802326],
               [-0.07017815]])

In [21]:
training_examples = get_training_example(words, 2, word2Ind, V)

In [22]:
x_array, y_array = next(training_examples)

In [23]:
x = x_array.reshape((V,1))
y = y_array.reshape((V,1))

In [24]:
z1 = np.dot(W1, x) + b1
h = relu(z1)
z2 = np.dot(W2, h) + b2
y_hat = softmax(z2)

In [25]:
def cross_entropy_loss(y_predicted, y_actual):
    loss = np.sum(-np.log(y_predicted)*y_actual)
    return loss

In [26]:
loss = cross_entropy_loss(y_hat,y)
loss

1.4650152923611106

In [27]:
grad_b2 = y_hat - y
grad_W2 = np.dot(y_hat - y, h.T)
grad_b1 = relu(np.dot(W2.T, y_hat - y))
grad_W1 = np.dot(relu(np.dot(W2.T, y_hat - y)), x.T)

In [28]:
alpha = 0.03

In [29]:
W1 = W1 - alpha * grad_W1
b1 = b1 - alpha * grad_b1
W2 = W2 - alpha * grad_W2
b2 = b2 - alpha * grad_b2

### Option 1

In [30]:
for word in word2Ind:
    word_embedding_vector = W1[:, word2Ind[word]]
    print(f'{word}: {word_embedding_vector}')

am: [0.41687358 0.32735501 0.26509758]
because: [ 0.08854191  0.22795148 -0.2397473 ]
happy: [-0.23495225 -0.23951958 -0.37770863]
i: [ 0.28320538  0.4117634  -0.11655134]
learning: [ 0.41800106 -0.23924344  0.34008124]


### Option 2

In [31]:
for word in word2Ind:
    word_embedding_vector = W2[word2Ind[word],:]
    print(f'{word}: {word_embedding_vector}')

am: [-0.22384758 -0.43362588  0.13310965]
because: [0.08265956 0.0775535  0.1772054 ]
happy: [ 0.19557112 -0.04637608 -0.1790735 ]
i: [ 0.06855622 -0.02363691  0.36107434]
learning: [ 0.33251813 -0.3982269  -0.43959196]


### Option 3

In [32]:
W3 = 0.5 * (W1.T + W2)
for word in word2Ind:
    word_embedding_vector = W3[word2Ind[word],:]
    print(f'{word}: {word_embedding_vector}')

am: [ 0.096513   -0.05313543  0.19910362]
because: [ 0.08560074  0.15275249 -0.03127095]
happy: [-0.01969057 -0.14294783 -0.27839106]
i: [0.1758808  0.19406324 0.1222615 ]
learning: [ 0.3752596  -0.31873517 -0.04975536]
